In [ ]:
# 텐서 연습

import tensorflow as tf

텐서1 = tf.constant([3,4,5])
텐서2 = tf.constant([6,7,8])
텐서3 = tf.constant([ [1,2], [3,4] ])

# print(tf.add(텐서1, 텐서2))
# print(tf.subtract(텐서1, 텐서2))
# print(tf.multiply(텐서1, 텐서2))
# print(tf.divide(텐서1, 텐서2))

# 특정 파라미터만 담긴 텐서 만들기

tensor = tf.zeros(10)
print("tensor :\n", tensor)
tensor = tf.zeros( [2,2] )
print("tensor :\n", tensor)
tensor = tf.zeros( [2,2,3] )
print("tensor :\n", tensor)

tensor = tf.constant([ [1,2,3], [3,4,5] ])

print("\n", tensor.shape, "\n")

w = tf.Variable(1.0)
print(w)
print(w.numpy())
w.assign(2)
print(w)
print(w.numpy())


tensor :
 tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float32)
tensor :
 tf.Tensor(
[[0. 0.]
 [0. 0.]], shape=(2, 2), dtype=float32)
tensor :
 tf.Tensor(
[[[0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]]], shape=(2, 2, 3), dtype=float32)

 (2, 3) 

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.0>
1.0
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.0>
2.0


In [ ]:
# y = ax + b
# 신발 = a*키 + b

# 키 = [170, 180, 175, 160]
# 신발 = [260, 270, 265, 255]

키 = 170
신발 = 260

#신발 = 키 * a + b

a = tf.Variable(0.1)
b = tf.Variable(0.2)

def 손실함수():
  #return tf.square(실제값 - 예측값)
  예측값 = (키 * a + b)
  return tf.square(260 - 예측값)

# 경사하강법
opt = tf.keras.optimizers.Adam(learning_rate= 0.1) # learning_rate = 얼마만큼 한꺼번에 업데이트 할 것인가?
# opt.minimize(손실함수, var_list=[a,b]) # var_list : 경사하강법으로 업데이트할 weight variable 목록 # Adam에서 minimize() 함수가 더이상 지원되지 않음

for i in range(300):
    with tf.GradientTape() as tape:
        손실 = 손실함수()
    gradients = tape.gradient(손실, [a, b]) # gradients : 기울기
    opt.apply_gradients(zip(gradients, [a, b])) # 기울기를 통해 weight variable 목록 업데이트

print(a)
print(b)

키 * a + b

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.5198832>
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.6198832>


<tf.Tensor: shape=(), dtype=float32, numpy=260.0>

In [1]:
# 대학 합격 예측 API 만들기

import pandas as pd
import numpy as np
import tensorflow as tf

data = pd.read_csv('gpascore.csv')
print(data)

#print(data.isnull().sum()) # 빈칸 정보를 출력

print("미국 대학 합격률을 예측하겠습니다.\n")
input_gre = input("- GRE 점수 : ")
input_gpa = input("- GPA 점수 : ")
input_rank = input("- 교내 순위 : ")

resultNdarray = model.predict( np.array( [ [ float(input_gre), float(input_gpa), float(input_rank)] ] ) )
result = math.floor(resultNdarray[0].item() * 100)
print("\n합격률은 %d%%입니다." % (result))


     admit    gre   gpa  rank
0        0  380.0  3.21     3
1        1  660.0  3.67     3
2        1  800.0  4.00     1
3        1  640.0  3.19     4
4        0  520.0  2.33     4
..     ...    ...   ...   ...
421      1  750.0  3.44     4
422      1  710.0  3.66     4
423      0  450.0  3.25     4
424      1  760.0  3.76     2
425      1  710.0  3.82     3

[426 rows x 4 columns]
미국 대학 합격률을 예측하겠습니다.

- GRE 점수 : 800
- GPA 점수 : 3.9
- 교내 순위 : 2


NameError: name 'model' is not defined

In [2]:
# torch 사용

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

data = pd.read_csv('gpascore.csv')
data = data.dropna()

y_data = data['admit'].values
x_data = []

for _, rows in data.iterrows():
    x_data.append([rows['gre'], rows['gpa'], rows['rank']])

x_data = torch.tensor(x_data, dtype=torch.float32)
y_data = torch.tensor(y_data, dtype=torch.float32).unsqueeze(1)

class AdmissionPredictor(nn.Module):
    def __init__(self):
        super(AdmissionPredictor, self).__init__()
        self.fc1 = nn.Linear(3, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

model = AdmissionPredictor()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(500):
    model.train()
    optimizer.zero_grad()
    outputs = model(x_data)
    loss = criterion(outputs, y_data)
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/500], Loss: {loss.item():.4f}')

with torch.no_grad():
    test_data = torch.tensor([[750.0, 3.70, 3.0], [400.0, 2.2, 1.0]], dtype=torch.float32)
    predictions = model(test_data)
    print(predictions)

import math
print(math.floor(predictions[0].item() * 100))
print(math.floor(predictions[1].item() * 100))

print("\n미국 대학 합격률을 예측하겠습니다.\n")
input_gre = float(input("- GRE 점수 : "))
input_gpa = float(input("- GPA 점수 : "))
input_rank = float(input("- 교내 순위 : "))

with torch.no_grad():
    user_input = torch.tensor([[input_gre, input_gpa, input_rank]], dtype=torch.float32)
    result = model(user_input)
    result = math.floor(result.item() * 100)
    print(f"\n합격률은 {result}%입니다.")


Epoch [100/500], Loss: 0.6917
Epoch [200/500], Loss: 0.6917
Epoch [300/500], Loss: 0.6917
Epoch [400/500], Loss: 0.6917
Epoch [500/500], Loss: 0.6917
tensor([[0.4729],
        [0.4729]])
47
47

미국 대학 합격률을 예측하겠습니다.

- GRE 점수 : 800
- GPA 점수 : 3.9
- 교내 순위 : 2

합격률은 47%입니다.
